<h1> Train DL Model 

<b> Libraries

In [ ]:
from tensorflow import cast, float32
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os




INIT_LR = 1e-4 # initial learning rate
EPOCHS = 20 # number of epochs to train for
BS = 32 # batch size

<b> Unzip dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#!unzip "/content/drive/My Drive/dataset.zip" -d "/content/drive/My Drive/Colab Notebooks/"

<b>Datasets loading

In [4]:
# load the dataset and categorize it
DIRECTORY = "/content/drive/My Drive/Colab Notebooks/dataset"
CATEGORIES = ["with_mask", "without_mask"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('start at ', datetime.now().strftime("%H:%M:%S"))
print("[INFO] start loading images... ")

data = []
data = np.array(data)
labels = []
count = 0
counted = 130000

# this part was done bacause the dataset is very huge. it contains more than 130K images.
# so, we saved it in multiple text files because the system crashes after loading every 8000-9000 images

# load a small part if the dataset and convert the images into float32
# then save the small part of the dataset in csv file
# and repeat running the same process until we load the whole dataset
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        if (count > counted and count < (counted +7000)):
            img_path = os.path.join(path, img)
            image = load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)

            data = np.append(data, cast(x=image, dtype=float32))
            labels.append(category)
        
        count += 1

        if count % 1000 == 0 and count > (counted - 1000):
            print(count, '  images are processed at  ', datetime.now().strftime("%H:%M:%S")) 

# save the processed part of the dataset as csv file to import it as text file later
np.savetxt("/content/drive/My Drive/Colab Notebooks/imagesdata18.csv", data, delimiter=",")
np.savetxt("/content/drive/My Drive/Colab Notebooks/labelsdata18.csv", labels, delimiter=",", fmt='%s')
print('images and labels data are saved at ', datetime.now().strftime("%H:%M:%S"))

start at  02:47:54
[INFO] start loading images... 
130000   images are processed at   02:48:15
131000   images are processed at   02:51:23
132000   images are processed at   03:03:16
133000   images are processed at   03:25:22
134000   images are processed at   03:53:33
135000   images are processed at   04:28:11
136000   images are processed at   05:08:41
137000   images are processed at   05:51:18
images and labels data are saved at  07:08:08


In [5]:
print('total images are:  ', count)

total images are:   137607


**Load the saved arrays**

In [3]:
# load the arrays from the saved csv files and prin the loading time after each loaded file
print('start at ', datetime.now().strftime("%H:%M:%S"))
imagesDataPart0 = np.loadtxt("/content/drive/My Drive/Colab Notebooks/imagesdata.csv", delimiter=",")
print('part 0 ended at ', datetime.now().strftime("%H:%M:%S"))

start at  08:52:59
part 0 ended at  10:07:11


In [5]:
print('start at ', datetime.now().strftime("%H:%M:%S"))
imagesDataPart1 = np.loadtxt("/content/drive/My Drive/Colab Notebooks/imagesdata1.csv", delimiter=",")
print('part 1 ended at ', datetime.now().strftime("%H:%M:%S"))

start at  10:16:58
part 1 ended at  11:18:20


In [ ]:
print('start at ', datetime.now().strftime("%H:%M:%S"))
imagesDataPart2 = np.loadtxt("/content/drive/My Drive/Colab Notebooks/imagesdata2.csv", delimiter=",")
print('part 2 ended at ', datetime.now().strftime("%H:%M:%S"))

start at  11:24:16


In [ ]:
print('start at ', datetime.now().strftime("%H:%M:%S"))
imagesDataPart3 = np.loadtxt("/content/drive/My Drive/Colab Notebooks/imagesdata3.csv", delimiter=",")
print('part 3 ended at ', datetime.now().strftime("%H:%M:%S"))

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

#data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

<b>Data agumentation

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

<b> Model training

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")

<b> Training loss and accuracy

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")